# 🧹 Task 1 — Data Collection & Preprocessing
### Customer Experience Analytics for Fintech Apps
**Commercial Bank of Ethiopia (CBE) • Bank of Abyssinia (BOA) • Dashen Bank**  
*November 2025*

## Objective
Scrape **minimum 1,200 real user reviews** (≥400 per bank) from the Google Play Store and deliver a **clean, analysis-ready dataset** with the following columns:

| Column   | Description                  |
|----------|------------------------------|
| `review` | User review text             |
| `rating` | 1–5 star rating              |
| `date`   | Review date (YYYY-MM-DD)     |
| `bank`   | CBE / BOA / DASHEN           |
| `source` | Google Play Store            |

___

### 1. Setup & Data Loading

In [1]:
import sys
import os

# Add project root (one directory above "notebooks")
sys.path.append(os.path.abspath(".."))

In [2]:
# import necessary modules and libraries
import pandas as pd
import re

from scripts.scrape_reviews import scrape_reviews_for_app
from scripts.preprocess_reviews import preprocess_pipeline

In [3]:
# package names for each application
APPS = {
    "CBE": "com.combanketh.mobilebanking",  #https://play.google.com/store/apps/details?id=com.combanketh.mobilebanking&hl=en
    "BOA": "com.boa.boaMobileBanking",    #https://play.google.com/store/apps/details?id=com.boa.boaMobileBanking&pcampaignid=web_share
    "Dashen": "com.dashen.dashensuperapp",  #https://play.google.com/store/apps/details?id=com.dashen.dashensuperapp&pcampaignid=web_share
}

### 2. Data Collection (Web Scraping)

In [4]:
all_reviews = [] 
# loop through each bank using the predefined APPS dictionary
for bank, package in APPS.items():
    data = scrape_reviews_for_app(bank, package)
    all_reviews.extend(data)

🔹 Scraping CBE...
✅ Finished CBE (600 reviews)
🔹 Scraping BOA...
✅ Finished BOA (600 reviews)
🔹 Scraping Dashen...
✅ Finished Dashen (600 reviews)


In [5]:
df = pd.DataFrame(all_reviews)  #Create a pandas DataFrame for easy manipulation and analysis
df.head()  #quick check

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,bank
0,28f229b5-0026-41b9-a1eb-b76e74736f63,TOMIZ Creativity,https://play-lh.googleusercontent.com/a-/ALV-U...,Make it user friendly.,2,0,None,2025-11-29 08:17:45,None,NaT,None,CBE
1,68d8daea-db47-4e23-a692-755173dea983,Tesfaye Abdi,https://play-lh.googleusercontent.com/a-/ALV-U...,maaliif daddafee install gaafata,3,0,5.2.1,2025-11-28 13:36:32,None,NaT,5.2.1,CBE
2,ee0dbb0e-4eb0-47b5-9874-c37877493f99,Betelhem Kebede,https://play-lh.googleusercontent.com/a/ACg8oc...,good app,5,0,None,2025-11-28 11:33:17,None,NaT,None,CBE
3,5112423d-e618-44ba-ba49-62677cb76cd6,Gurawaa Dooguu,https://play-lh.googleusercontent.com/a-/ALV-U...,This application is very important and advanta...,5,0,None,2025-11-27 23:08:20,None,NaT,None,CBE
4,bcb34681-1dd4-4781-b400-4393bb10b1d9,Yohannesb Adugna,https://play-lh.googleusercontent.com/a/ACg8oc...,why didn't work this app?,1,0,5.2.1,2025-11-27 22:40:35,None,NaT,5.2.1,CBE


In [6]:
df.to_csv("../data/raw_bank_reviews.csv", index=False)
print("💾 Saved to data/raw_bank_reviews.csv")


💾 Saved to data/raw_bank_reviews.csv


### 3. Preprocessing

In [7]:
# Work on a copy to preserve raw data in memory
df_clean = df.copy()
# Run the full pipeline defined in scripts/preprocess_reviews.py
df_clean = preprocess_pipeline(df_clean)


Starting preprocessing pipeline...

Raw data loaded: 1,800 reviews
🔹 Removed 0 duplicate reviews.
🔹 Removed 0 empty reviews.
Removed 253 reviews with non-Latin characters (Amharic/Arabic/etc)
→ Kept 1,547 clean English reviews for accurate sentiment analysis
Date normalized → datetime64[ns]
🔹 Standardized bank names.
🔹 Selected required final columns.
Balancing dataset to exactly 450 reviews per bank...

CBE: 521 → downsampled to 450
BOA: 514 → downsampled to 450
DASHEN: 512 → downsampled to 450

FINAL BALANCED DATASET: 1,350 reviews
bank
CBE       450
BOA       450
DASHEN    450


### 4. Validation Checks

In [8]:
# Check total review count
print("Total reviews:", len(df_clean))
# check for normalized date
df_clean.info()

Total reviews: 1350
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1350 entries, 0 to 1349
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   review  1350 non-null   object        
 1   rating  1350 non-null   int64         
 2   date    1350 non-null   datetime64[ns]
 3   bank    1350 non-null   object        
 4   source  1350 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 52.9+ KB


In [9]:
# Check distribution across banks
print(df_clean["bank"].value_counts())

bank
CBE       450
BOA       450
DASHEN    450
Name: count, dtype: int64


In [10]:
# Check missing values (< 5% expected)
df_clean.isnull().mean() * 100

review    0.0
rating    0.0
date      0.0
bank      0.0
source    0.0
dtype: float64

In [11]:
# Confirm final columns
df_clean.columns

Index(['review', 'rating', 'date', 'bank', 'source'], dtype='object')

In [12]:
# check for non latin words which could affect output in sentiment analysis
non_latin_left = df_clean["review"].fillna("").astype(str).str.contains(r'[^\x00-\x7F]', regex=True).sum()
print(f"Non-Latin characters still present: {non_latin_left}  → should be 0")
if non_latin_left == 0:
    print("Perfect! All non-Latin script removed.")
else:
    print("Warning: Some non-Latin text still exists!")

Non-Latin characters still present: 0  → should be 0
Perfect! All non-Latin script removed.


In [13]:
# save cleaned data set into data/processed
df_clean.to_csv("../data/processed/cleaned_reviews.csv", index=False)
print("💾 Saved cleaned dataset to ../data/cleaned_reviews.csv")

💾 Saved cleaned dataset to ../data/cleaned_reviews.csv
